# WIP Development of MPChecker: 004
 - 20190315
 - MJP
 - Initial look at MPChecker-like functionality 
 - Accuracy of positions returned by "OrbitObjectsInPointing" functionality 

In [1]:
%load_ext autoreload
%matplotlib inline

# Standard imports 
import numpy as np 
import matplotlib.pyplot as plt 
import healpy as hp
import sys, os

import MPC_library as MPCL
import phys_const as PHYS
import mpchecker 
import classes as Classes
import params 
import angles as ANG 


/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Some of the results returned below will depend on the assumed HP scale for the preprocessing
 - E.g. the HP # for the center of the pointing ...
 - At the time of evaluation the assumed parameters are ...

In [2]:
print("Healpix: Nside", params.hp_nside)
print("Healpix: Npix", params.hp_npix)
print("Healpix: Area [deg^2]", params.hp_area)
print("Healpix: Side [deg]", params.hp_side)

Healpix: Nside 256
Healpix: Npix 786432
Healpix: Area [deg^2] 0.05245585282569793
Healpix: Side [deg] 0.22903242745449373


# Set up a list of "fake" pointings for PanSTARRS (F51) 
 - These are all on the same night, and this night is the night of one of the nbody epochs, so no significant propogation is involved

In [16]:
# Overall / average quantities 
obsCode   = 'F51'
JDutc_INT = 2458360
FOV = 3.0 ## <<-- Assuming all pointings correspond to exposures with diameter = 3.0 degrees

# Generate ~10 randomly selected pointings 
Np = 10 
rInt = np.random.choice(params.hp_npix, size=Np, replace=False)
uv_ = [hp.pix2vec(params.hp_nside, pix, nest=params.hp_nested) for pix in rInt]
HP_ = [hp.vec2pix(params.hp_nside, *UV, nest=params.hp_nested) for UV  in uv_]
'''print("# HP: ", len(HP_), len(list(set(HP_))))
print("Before, HP_ = ", HP_[0])'''
print("Before, uv_ = ", uv_[:3])

# I don't want the pointings to be in the center of each HP
# - Randomly shift by an amount which keeps them within HP
RA_, Dec_ = ANG.unit2radec(uv_)
uv_randomized = []
for i, _ in enumerate(zip(RA_, Dec_ , uv_)):
    condition = True
    while condition:
        # Generating a random shit within the approx scale of a healpix
        dR = (params.hp_side/2.)*np.random.random()
        dD = (params.hp_side/2.)*np.random.random()
        sR = +1 if np.random.random() > 0.5 else -1
        sD = +1 if np.random.random() > 0.5 else -1
        # Shift the RA,Dec
        R = _[0]+sR*dR
        D = _[1]+sD*dD
        # Get the corresponding shifted UV & HP
        UV= ANG.radec2unit(R,D)
        HP= hp.vec2pix(params.hp_nside, *UV, nest=params.hp_nested)
        # Check that the HP is the same as it was initially
        # (I'm just trying to shift the center within the same HP)
        if HP == HP_[i]:
            condition = False 
            uv_randomized.append(UV) 
uv_ = np.array(uv_randomized)
print("After , uv_ = ", uv_[:3])

# Generate multiple different pointings 
PointingList = []
for i in range(Np):
    JDutc = JDutc_INT + 0.25 + 0.5*np.random.rand()
    RA , Dec  = ANG.unit2radec(uv_[i])
    
    # Make the Pointing ... 
    P = Classes.Pointing(obsCode, JDutc, RA[0] , Dec[0], FOV)
    
    # Calc & store observatory posn w.r.t. the ecliptic for the pointing 
    P.calc_heliocentric_position_of_observatory_in_ecliptic_coords()

    # Store in a list 
    PointingList.append( P )

# Examine some of the quantities to check that they are all as expected
print("Number of pointings generated = %d " % len(PointingList) )
print("First pointing: " , PointingList[0])
print("Last  pointing: " , PointingList[-1])
tmpR = [P.RADEC[0] for P in PointingList]
tmpD = [P.RADEC[1] for P in PointingList]
print("RA : min,median,max=",np.min(tmpR), np.median(tmpR), np.max(tmpR))
print("DEC: min,median,max=",np.min(tmpD), np.median(tmpD), np.max(tmpD))

Before, uv_ =  [(0.9459998234466996, -0.29331652460232316, -0.13802083333333331), (0.20742102332533008, 0.20742102332533002, -0.9560089111328125), (0.03491712718594951, 0.6840574609465695, 0.7285919189453125)]
After , uv_ =  [[ 0.94636721 -0.29253583 -0.13715643]
 [ 0.2073256   0.20689902 -0.95614272]
 [ 0.03464922  0.68441535  0.72826854]]
Number of pointings generated = 10 
First pointing:  F51_2458360.367764716_342.8228627014382_-7.8833342114836515
Last  pointing:  F51_2458360.290517107_276.26182559040865_48.08437379907602
RA : min,median,max= 44.940995435707364 102.95716431204488 342.8228627014382
DEC: min,median,max= -72.96827402368362 8.289166153462677 56.619114570716945


# Testing "OrbitObjectsInPointing"
 - A general function to perform the above methods of finding all objects within a FoV
 - This does all of the behind-the-scene calculations:
  - It calcs the HP for the fov
  - It gets the approx list of objects (preprocessed to night center)
  - It advances the approx list to the time of the pointing 
  - It selects the objects that are within the FoV

### First call to a pointing for the night is *SLOW*

In [17]:
%%time
OrbitObjectList, RA_, DEC_ = mpchecker.OrbitObjectsInPointing( PointingList[0] , '500' )
print("Pointing RA,DEC: ", PointingList[0].RADEC)
print("Pointing  JDutc: ", PointingList[0].JDutc)

Pointing RA,DEC:  (342.8228627014382, -7.8833342114836515)
Pointing  JDutc:  2458360.367764716
CPU times: user 9.09 ms, sys: 47.5 ms, total: 56.6 ms
Wall time: 136 ms


##### Explicitly examine the details of the first OrbitObject returned ...

In [18]:
print(len(OrbitObjectList), len(RA_), len(DEC_))
# Basic properties loaded into the OrbitObject ...
OO = OrbitObjectList[0]
print(OO.desig)
print(OO.epoch)
print(OO.GMcenter)
print(OO.cart_state.x, OO.cart_state.y, OO.cart_state.z, OO.cart_state.xd, OO.cart_state.yd, OO.cart_state.zd)

# Position propagated to the time of the pointing ... 
print(RA_[:1])
print(DEC_[:1])



96 96 96
c2449
2458360.5
0.00029591220828559115
3.1075548958891206 -1.1977399386908059 -0.06544894369868177 -0.0026292966748246655 -0.00873079238589724 0.0009992567439580915
[342.27851402]
[-9.26674808]


# Examine the locations of the OrbitObjects in OrbitObjectList
 - "d5155" == "395155"
 - Main-Belt Asteroid: https://www.minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id=395155
 - Using one-off JPL-Horizons query to orientate myself for now ...

# Explicit examination of data for d5155
 - Making sure that all stages of the data ingest & interpretation are being performed correctly